In [ ]:
#METHOD 1
#where we use dataframe and output of HTML is sorted by user story title
#_o16DXo6sQoWxWjeq9KtWGh9FNKhgSEF7l1obCahm9eM


#Boiler Plate for Rally Python Integration
import pyral
import sys
from pyral import Rally,rallyWorkset
import pandas as pd

options = [opt for opt in sys.argv[1:]if opt.startswith('--')]
server, user, password, apikey, workspace, project = rallyWorkset(options)
rally = Rally('rally1.rallydev.com',apikey=<key>
workspace=<workspace>, project=<project>)
#rally.enableLogging('rally.simple-use.log')
#rally.enableLogging('mypyral.log')


# List of US in a Release
relName='CLIP Release 49.0 (Apr 2021)'
rel='***** CLIP Release 49.0 (Apr 2021)'
release_req = rally.get('Release', fetch=True, query='Name = "%s"' % (rel))

rel = release_req.next()
#print(rel.Name)
user_stories = rally.get('HierarchicalRequirement', fetch=True, query='Release = %s' % (rel.ref))


#for us in user_stories:
    #if us.c_UserStoryType=='Business':
        #print(us.FormattedID,us.Name,us.oid)
        
        #write in excel
import xlsxwriter

workbook = xlsxwriter.Workbook('{}.xlsx'.format(relName))
worksheet = workbook.add_worksheet()
worksheet.write(0, 0,"User Story")
worksheet.write(0, 1,"Title")
worksheet.write(0, 2,"oid")
row = 1
col = 0

for us in user_stories:
    if us.c_UserStoryType=='Business':
        worksheet.write_string(row, col,us.FormattedID)
        worksheet.write_string(row, col + 1,  us.Name )
        worksheet.write(row, col + 2,  us.oid )
        #rallyURLid.append(us.oid)
        row += 1
        col = 0
        #print(us.FormattedID,us.Name,us.oid)

workbook.close()

# making data frame from csv file and sorting it from A to Z based on user story title
myDataFrame = pd.read_excel('{}.xlsx'.format(relName)).sort_values(by=['Title'])

#sorting data frame
#myData=data.sort_values(by=['Title'])

#write into HTML from DataFrame
fileout = open('{}.html'.format(relName), "w")

table = "<table border=1 dir=ltr style=width: 1089px;>\n"
anchorTag="<a href='https://rally1.rallydev.com/#/2275681671d/detail/userstory/{}'>"

#Create the table's column headers
table += "  <tr>\n"
for x in range(0,2):
    if x==0:
        table += "    <th>"+'User Story'+"</th>\n"
    else:
        table += "    <th>"+'Description'+"</th>\n"
table += "  </tr>\n"

# Create the table's row data
for row in range (1,len(myDataFrame)):
    table += "  <tr>\n"
    for col in range(0,2):
        if col==0:
            table += "    <td height=20 width=500>"+anchorTag.format(myDataFrame.iloc[row][2])+myDataFrame.iloc[row][0]+"</a></td>\n"
        elif col==1:
            table += "    <td height=20 width=500>"+myDataFrame.iloc[row][1]+"</td>\n"
        else:
            break
    table += "  </tr>\n"

table += "</table>"

fileout.writelines(table)
fileout.close()

In [ ]:
#METHOD 2
#where user story list in HTML is not sorted
#Boiler Plate for Rally Python Integration
import pyral
import sys
from pyral import Rally,rallyWorkset

# Rally connection string
#rally.enableLogging('rally.simple-use.log')
#rally.enableLogging('mypyral.log')


# List of US in a Release
rel='CLIP Release 42.0 (Feb 2020)'
release_req = rally.get('Release', fetch=True, query='Name = "%s"' % (rel))

rel = release_req.next()
#print(rel.Name)
user_stories = rally.get('HierarchicalRequirement', fetch=True, query='Release = %s' % (rel.ref))


#for us in user_stories:
    #if us.c_UserStoryType=='Business':
        #print(us.FormattedID,us.Name,us.oid)
        
        #write in excel
import xlsxwriter

workbook = xlsxwriter.Workbook('Demo.xlsx')
worksheet = workbook.add_worksheet()

row = 1
col = 0
rallyURLid=[]

for us in user_stories:
    if us.c_UserStoryType=='Business':
        worksheet.write_string(row, col,us.FormattedID)
        worksheet.write_string(row, col + 1,  us.Name )
        #worksheet.write(row, col + 2,  us.oid )
        rallyURLid.append(us.oid)
        row += 1
        col = 0
        #print(us.FormattedID,us.Name,us.oid)

workbook.close()

#read from excel
import xlrd

wb=xlrd.open_workbook('Demo.xlsx')
sheet=wb.sheet_by_index(0)
sheet.cell_value(0,0)
v=[]
l=[]
for i in range(sheet.nrows):
    for j in range(sheet.ncols):
        v.append(sheet.cell_value(i,j))
        #print(sheet.cell_value(i,j))
    l.append(v)
    v=[]

#print(l)

#write into HTML
fileout = open("Demo.html", "w")

table = "<table border=1 dir=ltr style=width: 1089px;>\n"
anchorTag="<a href='https://rally1.rallydev.com/#/2275681671d/detail/userstory/{}'>"

# Create the table's column headers
table += "  <tr>\n"
for x in range(sheet.ncols):
    table += "    <th>"+sheet.cell_value(0,x)+"</th>\n"
table += "  </tr>\n"

# Create the table's row data
for row in range (1,sheet.nrows):
    table += "  <tr>\n"
    for c in range(0,sheet.ncols):
        if c==0:
            table += "    <td height=20 width=500>"+anchorTag.format(rallyURLid[row-1])+sheet.cell_value(row,c)+"</a></td>\n"
        else:
            table += "    <td height=20 width=500>"+sheet.cell_value(row,c)+"</td>\n"
    table += "  </tr>\n"

table += "</table>"

fileout.writelines(table)
fileout.close()

In [ ]:
#pull CFD for CLIP based on release


import sys
import requests
import pyral
from pyral import Rally,rallyWorkset
import pandas as pd
from requests.auth import HTTPBasicAuth
import xlsxwriter
import string

# Rally Connection string

#rally.enableLogging('rally.simple-use.log')
#rally.enableLogging('mypyral.log')


# List of US in a Release
relNameD='CLIP 49.0 CFD'
relD='***** CLIP Release 49.0 (Apr 2021)'
release_reqD= rally.get('Release', fetch=True, query='Name = "%s"' % (relD))

relD = release_reqD.next()
release_defects = rally.get('Defect', fetch=True, query='Release = %s' % (relD.ref))


#for us in user_stories:
    #if us.c_UserStoryType=='Business':
        #print(us.FormattedID,us.Name,us.oid)
        
#write in excel

workbook = xlsxwriter.Workbook('{}.xlsx'.format(relNameD))
worksheet = workbook.add_worksheet()
worksheet.write(0, 0,"Defect")
worksheet.write(0, 1,"Title")
worksheet.write(0, 2,"oid")
worksheet.write(0, 3,"Iteration")
worksheet.write(0, 4,"Severity")
worksheet.write(0, 5,"Submitted By")
row = 1
col = 0

for rd in release_defects:
    try:
        rd.__collection_ref_for_Tags
    except AttributeError:
          pass
    else:
        x=(rd.__collection_ref_for_Tags)#this code has link that contains JSON response for a defect
        tagResponse = requests.get(x, auth = HTTPBasicAuth('clip-rally@cisco.com', 'Clip!235')) 
        k=(tagResponse.json()["QueryResult"]["Results"][0].get('Name'))
        if k.find('CFD')!=-1:
            worksheet.write_string(row, col,rd.FormattedID)
            worksheet.write_string(row, col + 1,  rd.Name )
            worksheet.write(row, col + 2,  rd.oid )
            response2 = rally.get('Defect', fetch=True, query='FormattedID = %s'%(rd.FormattedID))
            story2 = response2.next()
            if story2.Iteration!=None:
                worksheet.write(row, col + 3,  story2.Iteration.Name)
            else:
                worksheet.write(row, col + 3, 'No Iteration Scheduled')
            worksheet.write(row, col + 4,  rd.Severity )
            worksheet.write(row, col + 5,  story2.SubmittedBy.DisplayName )
            #print(story1.SubmittedBy.DisplayName)
            #rallyURLid.append(us.oid)
            row += 1
            col = 0
        #if us.c_UserStoryType=='Business':
        
        #print(us.FormattedID,us.Name,us.oid)

workbook.close()
# making data frame from csv file and sorting it from A to Z based on user story title
myDataFrameD = pd.read_excel('{}.xlsx'.format(relNameD)).sort_values(by=['Iteration','Title'])

#print(myDataFrameD)
#sorting data frame
#myData=data.sort_values(by=['Title'])
#write into HTML from DataFrame
fileout = open('{}.html'.format(relNameD), "w")

tableDef = "<table border=1 dir=ltr style=width: 1089px;>\n"
anchorTagD="<a href='https://rally1.rallydev.com/#/2275681671d/detail/defect/{}'>"

#Create the table's column headers
tableDef += "  <tr>\n"
for x in range(0,5):
    if x==0:
        tableDef += "    <th>"+'Defect'+"</th>\n"
    elif x==1:
        tableDef += "    <th>"+'Description'+"</th>\n"
    elif x==2:
        tableDef += "    <th>"+'Iteration'+"</th>\n"
    elif x==3:
        tableDef += "    <th>"+'Severity'+"</th>\n"
    else:
        tableDef += "    <th>"+'Submitted By'+"</th>\n"
tableDef += "  </tr>\n"

# Create the table's row data
for row in range (0,len(myDataFrameD)):
    tableDef += "  <tr>\n"
    for col in range(0,5):
        if col==0:
            tableDef += "    <td height=20 width=500>"+anchorTagD.format(myDataFrameD.iloc[row][2])+myDataFrameD.iloc[row][0]+"</a></td>\n"
        elif col==1:
            tableDef += "    <td height=20 width=500>"+myDataFrameD.iloc[row][1]+"</td>\n"
        elif col==2:
            tableDef += "    <td height=20 width=500>"+myDataFrameD.iloc[row][3]+"</td>\n"
        elif col==3:
            tableDef += "    <td height=20 width=500>"+myDataFrameD.iloc[row][4]+"</td>\n"
        elif col==4:
            #below line sees if submitted by in rally is staring with letter or not.Like bernard has emoji before his name starts
            if myDataFrameD.iloc[row][5].split()[0][0] in (string.ascii_uppercase):
                tableDef += "    <td height=20 width=500>"+myDataFrameD.iloc[row][5]+"</td>\n"
            else:
                tableDef += "    <td height=20 width=500>"+myDataFrameD.iloc[row][5].split()[1]+"</td>\n"
        else:
            break
    tableDef += "  </tr>\n"

tableDef += "</table>"
fileout.writelines(tableDef)
fileout.close()